Separate file to download posts in the background and save them to the folder so that I can work on the rest of the project while it is happening

I did this before we started looking into BeautifulSoup. I don't want to mess with it now. Legacy Code!

In [1]:
import time, requests
import pandas as pd
import pixiedust

Pixiedust database opened successfully


In [2]:
#Max num of pages to try and get
posts=50

In [3]:

jsons = {'talesfromtechsupport':pd.DataFrame(), 'LFG':pd.DataFrame(), 'nosleep':pd.DataFrame()}
the_headers = {'User-agent': 'project3 Bot 0.4'}

In [4]:
features = [
    'subreddit',
    'url',
    'author',
    'domain',
    'downs',
    'is_self', 
    'is_video', 
    'likes',
    'media',
    'num_comments',
    'num_crossposts',
    'num_reports',
    'selftext',
    'score',
    'title',
    'ups'
]

In [5]:
%%time

# The meat of this file.  It goes through and gets posts+1 pages of posts from each subreddit, gets out the relevant features, adds them to a DataFrame and appends to that DataFrame until the loop finishes
for k, v in jsons.items():
    URL = "http://www.reddit.com/r/" + k + ".json"

    res = requests.get(URL, headers = the_headers)
    the_json = res.json()

    pre_temp_df = pd.DataFrame(the_json['data']['children'])
    pre_temp_df = pd.DataFrame(list(pre_temp_df['data']))[features]
    jsons[k] = pre_temp_df.copy()
    
    #print('a', pre_temp_df.shape)
    
    URL_EXTENDER = "?after="

    print(k, 'Iterations:')
    for i in range(posts): 
        last_title = the_json['data']['after']

        try:
        #retrieve new data
            if last_title==None:
                break
                
            temp_data = requests.get(URL+URL_EXTENDER+last_title, headers = the_headers)

            the_json = temp_data.json()
            
            #add to the temp df more
        
            pre_temp_df = pd.DataFrame(the_json['data']['children'])

            if (len(pre_temp_df.columns) == 0):
                break
            pre_temp_df = pd.DataFrame(list(pre_temp_df['data']))[features]
            #print('b', pre_temp_df.shape)

            jsons[k] = jsons[k].append(pre_temp_df, ignore_index = True)
            
            time.sleep(3)
            if i % 10 == 0:
                print()
            print('%02d' % (i+1), end=" ")

        except _ as e:
            print()
            print(e, "Let's see what we've got!")
            break
    
    #add temp df to main df
    print()
    print(k,' temp df shape', jsons[k].shape)
    print('\n\n\n')
    
#print(jsons)

talesfromtechsupport Iterations:

01 02 03 04 05 06 07 08 09 10 
11 12 13 14 15 16 17 18 19 20 
21 22 23 24 25 26 27 28 29 30 
31 32 33 34 35 36 37 38 39 
talesfromtechsupport  temp df shape (978, 16)




LFG Iterations:

01 02 03 04 05 06 07 08 09 10 
11 12 13 14 15 16 17 18 19 20 
21 22 23 24 25 26 27 28 29 30 
31 32 33 34 35 36 
LFG  temp df shape (921, 16)




nosleep Iterations:

01 02 03 04 05 06 07 08 09 10 
11 12 13 14 15 16 17 18 19 20 
21 22 23 24 25 26 27 28 29 30 
31 32 
nosleep  temp df shape (816, 16)




CPU times: user 3.83 s, sys: 309 ms, total: 4.14 s
Wall time: 5min 56s


In [6]:
print(last_title==None)

True


In [7]:
for k, v in jsons.items():
    print(v.shape)

(978, 16)
(921, 16)
(816, 16)


In [8]:
main_df = pd.DataFrame()
for k, v in jsons.items():
    main_df = main_df.append(v.copy(), ignore_index = True)
main_df.drop_duplicates(inplace = True, subset=['url'])
print(main_df.shape)

(2715, 16)


In [9]:
main_df.head()

,subreddit,url,author,domain,downs,is_self,is_video,likes,media,num_comments,num_crossposts,num_reports,selftext,score,title,ups
0,talesfromtechsupport,https://www.reddit.com/r/talesfromtechsupport/...,MagicBigfoot,self.talesfromtechsupport,0,True,False,None,None,77,0,None,"Hey, we can have two stickies now!\n\n---\n\nS...",1952,TFTS POSTING RULES (MOBILE USERS PLEASE READ!),1952
1,talesfromtechsupport,https://www.reddit.com/r/talesfromtechsupport/...,TheTableClothe,self.talesfromtechsupport,0,True,False,None,None,56,0,None,I did tech support for a large broadband compa...,509,My WiFi isn’t working!,509
2,talesfromtechsupport,https://www.reddit.com/r/talesfromtechsupport/...,miscreancy,self.talesfromtechsupport,0,True,False,None,None,155,0,None,Another tale from my days working for $GiantSu...,1625,"""It could be worse, this could be a company ma...",1625
3,talesfromtechsupport,https://www.reddit.com/r/talesfromtechsupport/...,I_Have_A_Chode,self.talesfromtechsupport,0,True,False,None,None,4,0,None,So this happened friday morning. \n\n$chode = ...,200,The mysterious missed phone calls!,200
4,talesfromtechsupport,https://www.reddit.com/r/talesfromtechsupport/...,kayakmainac,self.talesfromtechsupport,0,True,False,None,None,56,0,None,LTL/FTP\n\nSo I work in IT services for a univ...,493,A UPS test! During peak time? The day before a...,493


In [10]:
#save to csv
path = './data/new_data.csv'
main_df.to_csv(path, index=False)

In [ ]:
#test the csv
path = './data/new_data.csv'
test_db = pd.read_csv(path)
print(test_db.shape)
test_db.equals(main_df) #returns false because the Nones are now NaNs


In [ ]:
test_db.head()